In [2]:
# !pip install lbl2vec
from lbl2vec import Lbl2Vec

In [ ]:
# !python -m pip install --upgrade pip

In [ ]:
# !pip install wheel
# !python -m pip install -U pip setuptools

In [1]:
# !pip install pip==22.1.2
# !pip list

In [5]:
# !pip install fasttext

In [6]:
# import fasttext.util
# fasttext.util.download_model('en', if_exists='ignore')  # English
# ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
# import gensim.models as g

# # model="../../../../enwiki_dbow/doc2vec.bin"  #point to downloaded pre-trained doc2vec model
# model='../../../../cc.en.300.bin'
# #load model
# m = g.Doc2Vec.load(model)

# Keywords

In [9]:
conflict = ['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army']
politics = ['politics', 'debate', 'election', 'government', 'democracy']
fashion = ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit']
culture = ['culture', 'opera', "festival", 'collection', 'exhibition', 'art', 'museum']
sports = ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play']
education = ['education', 'professor', 'study', 'research', 'university']
# football = []
# other

descriptive_keywords = [conflict, politics, fashion, culture, sports, education]
# How to avoid words with high tf?

In [10]:
descriptive_keywords

[['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army'],
 ['politics', 'debate', 'election', 'government', 'democracy'],
 ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit'],
 ['culture', 'opera', 'festival', 'collection', 'exhibition', 'art', 'museum'],
 ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play'],
 ['education', 'professor', 'study', 'research', 'university']]

In [11]:
import os
import pickle

from collections import Counter

In [5]:
WORDLIST_LOCATION = '../../../../data/enwiki_city_pairs_lemmatised/NOUN_SPACY_LARGE_REAL_WORDS/biggest_cities_5'
wordlists = []
city_pairs = []

for root, dirs, files in os.walk(WORDLIST_LOCATION, topdown=True):
    for name in files:
        file_path = os.path.join(root, name)
        
        with open(file_path, 'rb') as fp:
            wordlists.append(pickle.load(fp))
            city_pairs.append(name.split('__')[1])

In [6]:
# Sort by number of words

data = list(zip(wordlists, city_pairs))
sorted_data = sorted(data, key=lambda x: len(x[0]), reverse=True)

for x in sorted_data[:20]:
    print(len(x[0]), x[1])


427850 paris_london
152094 london_berlin
151175 paris_berlin
61672 paris_milan
60544 paris_madrid
46708 london_milan
45255 london_madrid
22840 madrid_milan
19438 madrid_berlin
17341 berlin_milan


In [76]:
len(sorted_data[0][0]), sorted_data[0][1]
c = Counter(sorted_data[0][0])
most_common = c.most_common()

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm

In [13]:
GLOVE_LOCATION = '../../../../glove.42B.300d.txt'

In [14]:
embeddings_dict = {}

In [15]:
%%time # ~3 minutes to load

num_lines = sum(1 for line in open(GLOVE_LOCATION,'r', encoding="utf-8"))

with open(GLOVE_LOCATION, 'r', encoding="utf-8") as f:
     for line in tqdm(f, total=num_lines):
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

  0%|          | 0/1917495 [00:00<?, ?it/s]

CPU times: total: 3min 6s
Wall time: 3min 10s


In [16]:
descriptive_keywords

[['war', 'fight', 'death', 'hostility', 'bomb', 'battle', 'nazi', 'army'],
 ['politics', 'debate', 'election', 'government', 'democracy'],
 ['fashion', 'model', 'magazine', 'walk', 'glamour', 'outfit'],
 ['culture', 'opera', 'festival', 'collection', 'exhibition', 'art', 'museum'],
 ['sport', 'medal', 'game', 'championship', 'club', 'score', 'play'],
 ['education', 'professor', 'study', 'research', 'university']]

In [ ]:
# Calculate distance between two words

In [20]:
from scipy.spatial.distance import cosine

word_1 = embeddings_dict['fashion']
word_2 = embeddings_dict['war']
print(1 - cosine(word_1, word_2))

0.3141452968120575


In [145]:
words = [x[0] for x in most_common[:1000]]
print(words)

['art', 'year', 'work', 'time', 'city', 'exhibition', 'world', 'opera', 'school', 'music', 'company', 'artist', 'gallery', 'film', 'member', 'museum', 'festival', 'war', 'society', 'painting', 'family', 'collection', 'tour', 'service', 'life', 'book', 'study', 'performance', 'concert', 'group', 'fashion', 'career', 'role', 'day', 'office', 'father', 'theatre', 'house', 'woman', 'government', 'country', 'century', 'son', 'place', 'production', 'studio', 'orchestra', 'director', 'college', 'number', 'man', 'album', 'history', 'event', 'design', 'child', 'week', 'month', 'death', 'ballet', 'age', 'end', 'friend', 'university', 'club', 'portrait', 'business', 'series', 'song', 'period', 'professor', 'stage', 'wife', 'painter', 'magazine', 'band', 'state', 'people', 'academy', 'summer', 'symphony', 'daughter', 'student', 'success', 'award', 'title', 'home', 'amp', 'mother', 'king', 'project', 'brother', 'street', 'model', 'line', 'flight', 'debut', 'record', 'law', 'degree', 'air', 'version

In [89]:
textstring = "Napoleon as Mars the Peacemaker had its inception after Canova was hired to make a bust of Napoleon in 1802. The statue was begun in 1802, with Napoleon requesting to be shown in a French General's uniform, Canova rejected this, insisting on an allusion to Mars, the Roman god of War. It was completed in 1806. In 1811, the statue arrived in Paris, but not installed; neither was its bronze copy in the Foro Napoleonico in Milan. In 1815, the original went to the Duke of Wellington, after his victory at Waterloo against Napoleon. "

# words = ['peacemaker', 'inception', 'hire', 'bust', 'statue', 'begin', 'request', 'show', 'french', 'general', 'uniform', 'reject', 'insist', 'allusion', 'roman', 'war', 'complete', 'statue', 'arrive', 'instal', 'bronze', 'copy', 'original', 'go', 'victory']
words = ['peacemaker', 'inception', 'bust', 'statue', 'french', 'general', 'uniform', 'allusion', 'roman', 'war', 'statue', 'bronze', 'copy', 'original', 'victory']

In [83]:
# words = ['red', 'card', 'club', 'world', 'championship', 'necaxa', 'time', 'real', 'yellow', 'card', 'red', 'card', 'red', 'card', 'red', 'card', 'la', 'galaxy', 'home', 'defeat', 'sounder', 'red', 'card', 'injury', 'time', 'match', 'competitive', 'game', 'red', 'card', 'match', 'average']
#words = ['red', 'card', 'come', 'club', 'world', 'championship', 'necaxa', 'time', 'real', 'amass', 'yellow', 'card', 'red', 'card', 'receive', 'red', 'card', 'red', 'card', 'la', 'galaxy', 'come', 'home', 'defeat', 'sounder', 'receive', 'red', 'card', 'pick', 'injury', 'time', 'match', 'play', 'competitive', 'game', 'receive', 'red', 'card', 'match', 'average']

In [94]:
textstring = "David Robert Joseph Beckham (; born 2 May 1975) is an English former professional footballer, the current president &amp; co-owner of Inter Miami CF and co-owner of Salford City. He played for Manchester United, Preston North End (on loan), Real Madrid, AC Milan (on loan), LA Galaxy, Paris Saint-Germain and the England national team, for which he held the appearance record for an outfield player until 2016. He is the first English player to win league titles in four countries: England, Spain, the United States and France. He retired in May 2013 after a 20-year career, during which he won 19 major trophies."
words = ['bear', 'english', 'professional', 'footballer', 'current', 'president', 'amp', 'co', 'owner', 'co', 'owner', 'play', 'loan', 'loan', 'hold', 'appearance', 'record', 'outfield', 'player', 'english', 'player', 'win', 'league', 'title', 'country', 'retire', 'year', 'career', 'win', 'major', 'trophy']

In [154]:
textstring = "In 1872, Abbott went abroad to study with Antonio Sangiovanni in Milan. This was followed by further studies with Mathilde Marchesi, Pierre François Wartel and Enrico Delle Sedie in Paris. She appeared in several productions in Paris, earning rave reviews for her fine soprano voice. She was awarded a contract with the Royal Opera in London and made her début at Covent Garden as Marie in 'La Fille du régiment' in 1876. However, her contract was cancelled shortly thereafter when she refused to sing Violetta from Verdi's 'La Traviata' on moral grounds. That same year she secretly married Eugene Wetherell (d. 1889) and they returned to the United States, where she remained for the rest of her life."
words = ['go', 'study', 'follow', 'study', 'sedie', 'appear', 'production', 'earn', 'rave', 'review', 'fine', 'soprano', 'voice', 'award', 'contract', 'royal', 'opera', 'début', 'covent', 'garden', 'régiment', 'contract', 'cancel', 'refuse', 'sing', 'violetta', 'moral', 'ground', 'year', 'marry', 'return', 'remain', 'rest', 'life']

In [155]:
from pprint import pprint

In [215]:
%%time

verbose1, verbose2 = False, True
similar_categories = []
similar_categories_extended = []
for word in words:
    closeness = []
    if verbose1 or verbose2:
        print(f"word: \t\t{word}")
    for category in descriptive_keywords:
        summed_similarity = 0
        for keyword in category:
            keyword_vector = embeddings_dict[keyword]        
            word_vector = embeddings_dict[word]
            value = 1 - cosine(keyword_vector, word_vector)
            summed_similarity += value
            break
        
        normalized_similarity = summed_similarity/len(category)
        normalized_similarity = summed_similarity
        closeness.append((normalized_similarity, category[0]))
        
        if verbose1:
            print('___________________________')
            print('===>', '\tcategory:', category[0], '\n\tsimilarity:', normalized_similarity, f"\n\tkeywords: {category}", )
     
    similar_category = max(closeness)
    
    sortedcat = sorted(closeness, key=lambda item: item[0], reverse=True)
    if (sortedcat[0][0] - 0.05) > sortedcat[2][0]:
        allowed = True
        print(f"analysis: \tenough difference\nscores:")
        print("\t----category----          ----score----")
        for i in sortedcat:
            print(f"\t{i[1]:<10s} \t\t{i[0]}")
        print()
    else:
        allowed = False
        print(f"analysis: \tnot enough difference\nscores:")
        print("\t----category----          ----score----")
        for i in sortedcat:
            print(f"\t{i[1]:<10s} \t\t{i[0]}")
        print()
    if similar_category[0] > 0.1 and allowed:
        similar_categories.append((word, similar_category))
#         if (sortedcat[0][0] - 0.05 < sortedcat[2][0]):
#             similar_categories_extended.append((word, sortedcat[0:3]))
#         elif (sortedcat[0][0] - 0.05 < sortedcat[1][0]):
#             similar_categories_extended.append((word, sortedcat[0:2]))
#         else:
#             print('xxx', word, sortedcat[0])
#             similar_categories_extended.append((word, [sortedcat[0]]))
        if verbose2:
            #print('category similarity:')
            # pprint(sorted(closeness, key=lambda x: x[0], reverse=True))
            print()
            print(f"==> \tword: {word}, \n\tcategory:, {similar_category[1]}, \n\tsimilarity score:, {similar_category[0]}")
    else:
        reason = 'ambiguity' if not allowed else 'low similarity score'
        print(f"**left out due to {reason}**")
    

    print('='*100)
    print('\n')

print('\nwords:', words, '\n')

similar_categories

word: 		go
analysis: 	not enough difference
scores:
	----category----          ----score----
	war        		0.4399181008338928
	education  		0.4335162937641144
	sport      		0.4327922761440277
	culture    		0.4205860197544098
	fashion    		0.415298730134964
	politics   		0.38997897505760193

**left out due to ambiguity**


word: 		study
analysis: 	enough difference
scores:
	----category----          ----score----
	education  		0.574546754360199
	culture    		0.4654148817062378
	politics   		0.37829726934432983
	war        		0.3699481189250946
	fashion    		0.32155609130859375
	sport      		0.3135829269886017


==> 	word: study, 
	category:, education, 
	similarity score:, 0.574546754360199


word: 		follow
analysis: 	enough difference
scores:
	----category----          ----score----
	fashion    		0.43281760811805725
	education  		0.4069565534591675
	culture    		0.3794364929199219
	politics   		0.37496280670166016
	war        		0.3284284174442291
	sport      		0.32283997535705566


==> 

[('study', (0.574546754360199, 'education')),
 ('follow', (0.43281760811805725, 'fashion')),
 ('study', (0.574546754360199, 'education')),
 ('earn', (0.4088855981826782, 'education')),
 ('rave', (0.32954198122024536, 'culture')),
 ('review', (0.4184594452381134, 'education')),
 ('award', (0.47735095024108887, 'education')),
 ('contract', (0.3552599549293518, 'education')),
 ('covent', (0.18027999997138977, 'fashion')),
 ('garden', (0.3839989900588989, 'fashion')),
 ('contract', (0.3552599549293518, 'education')),
 ('ground', (0.41804057359695435, 'war')),
 ('year', (0.49935826659202576, 'education')),
 ('return', (0.406259685754776, 'war')),
 ('rest', (0.4607382118701935, 'war')),
 ('life', (0.5718253254890442, 'culture'))]

In [285]:
words = ["football", "club", "archery", "athlete", "ball", "biking", "student", 'catwalk', 'brainfreeze', 'nerd', 'geek', 
         'school', 'lawschool', 'helmet', 'poison', 'sunglasses', 'philosophy', 'brain']

In [229]:
categories_dict = {key[0]: 0 for key in descriptive_keywords}
categories_dict

{'war': 0,
 'politics': 0,
 'fashion': 0,
 'culture': 0,
 'sport': 0,
 'education': 0}

In [238]:
similar_categories
# similar_categories_extended

[('study', (0.574546754360199, 'education')),
 ('follow', (0.43281760811805725, 'fashion')),
 ('study', (0.574546754360199, 'education')),
 ('earn', (0.4088855981826782, 'education')),
 ('rave', (0.32954198122024536, 'culture')),
 ('review', (0.4184594452381134, 'education')),
 ('award', (0.47735095024108887, 'education')),
 ('contract', (0.3552599549293518, 'education')),
 ('covent', (0.18027999997138977, 'fashion')),
 ('garden', (0.3839989900588989, 'fashion')),
 ('contract', (0.3552599549293518, 'education')),
 ('ground', (0.41804057359695435, 'war')),
 ('year', (0.49935826659202576, 'education')),
 ('return', (0.406259685754776, 'war')),
 ('rest', (0.4607382118701935, 'war')),
 ('life', (0.5718253254890442, 'culture'))]

In [240]:
for x in similar_categories:
    categories_dict[x[1][1]] += x[1][0] #print(x[1])
    
categories_dict

# sorted(categories_dict, key=lambda )
results = list(sorted(categories_dict.items(), key=lambda item: item[1], reverse=True))
pprint(results)

print(f"\nThe dominant category is: '{results[0][0]}'", end='')
if (results[0][1] - (float(results[0][1])/5)) <= results[1][1]:  
    print(f", closely followed by: '{results[1][0]}'.")

[('education', 7.327335357666016),
 ('war', 2.5700769424438477),
 ('fashion', 1.994193196296692),
 ('culture', 1.802734613418579),
 ('politics', 0),
 ('sport', 0)]

The dominant category is: 'education'

In [146]:
for x in similar_categories_extended:
    #print(x)
    for category in x[1]:
        #print(category)
        categories_dict[category[1]] += category[0]
        # print(category)
    #categories_dict[x[1]]
    #print(x)

categories_dict

{'war': 3.8231630325317383,
 'politics': 0.46853357553482056,
 'fashion': 1.8761795163154602,
 'culture': 0.9031941294670105,
 'sport': 4.09539407491684,
 'education': 7.556927263736725}

In [50]:
words

['art',
 'year',
 'work',
 'time',
 'city',
 'exhibition',
 'world',
 'opera',
 'school',
 'music',
 'company',
 'artist',
 'gallery',
 'film',
 'member',
 'museum',
 'festival',
 'war',
 'society',
 'painting']

In [69]:
import fasttext

ModuleNotFoundError: No module named 'fasttext'

In [ ]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

vectors = load_vectors("../../../../cc.en.300.vec")

In [72]:
from gensim.models.wrappers import FastText

#model = FastText.load_fasttext_format('wiki.simple')

ModuleNotFoundError: No module named 'gensim.models.wrappers'

In [2]:
import gensim

In [3]:
path = "../../../../cc.en.300.vec"
model4 = gensim.models.fasttext.load_facebook_vectors(path)
# model3 = gensim.models.fasttext.load_facebook_model(path, encoding='utf-8')

ERROR! Session/line number was not unique in database. History logging moved to new session 70


NotImplementedError: Supervised fastText models are not supported

In [68]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors, Word2Vec

# wv_from_bin = KeyedVectors.load_word2vec_format(datapath("cc.en.300.bin"), binary=True)  # C bin forma
# word_vectors = KeyedVectors.load("../../../../cc.en.300.bin")
word_vectors2 = Word2Vec.load("../../../../cc.en.300.bin")
# wv_from_bin.save("cc_en_300.txt")

UnpicklingError: invalid load key, '\xba'.

In [60]:
from gensim.models import word2vec
model2 = word2vec.Word2Vec.load_word2vec_format("..//..//..//..//cc.en.300.bin", binary=True)
model2.save("cc_en_300.txt")

AttributeError: type object 'Word2Vec' has no attribute 'load_word2vec_format'

In [10]:
# init model #pretrained_w2v_model
model = Lbl2Vec(keywords_list=descriptive_keywords, tagged_documents=None, doc2vec_model=wv_from_bin)
# train model
model.fit()

NameError: name 'd' is not defined

In [ ]:
# get similarity scores for each new document from trained model
model.predict_new_docs(tagged_docs=tagged_docs)